# Домашнее задание 4

Максимальный балл за домашнюю работу 10.
Необходимо выполнять все задания в ноутбуке с помощью чистого SQL и SQLAlchemy ORM:
- осуществить подключение к БД
- каждое задание оформить в отдельной ячейке
- результат выполнения запросов записать в pandas dataframe

In [ ]:
Пока что сделал только на чистом SQL, c SQLAlchemy ORM какие то проблемы 
возниколи, но я разбираюсь

In [1]:
!pip install psycopg2

In [17]:
import psycopg2
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [50]:
connection = psycopg2.connect(
    host='ep-bitter-breeze-a2oczi1e.eu-central-1.aws.neon.tech',
    database='club',
    user='KotelnikovNA',
    password='9YRzhyH2txkK'
    )

## Задание 1 (1 балл)

Сформируйте упорядоченный список из первых 10 фамилий в таблице участников?  
Список не должен содержать дубликатов.

In [54]:
cursor = connection.cursor()
sql = '''
select distinct surname
from club.cd.members m
order by surname
limit 10
'''
with cursor:
    with connection.cursor() as cur:
        cur.execute(sql)
        result = pd.DataFrame(cur.fetchall())
result

,0
0,Bader
1,Baker
2,Boothe
3,Butters
4,Coplin
5,Crumpet
6,Dare
7,Farrell
8,GUEST
9,Genting


## Задание 2  (1 балл)

Выведите имя и фамилию последнего зарегистрировавшегося участника, а также дату.

In [40]:
sql = '''
select firstname, surname, joindate
from club.cd.members m
order by joindate desc
limit 1
'''
with cursor:
    with connection.cursor() as cur:
        cur.execute(sql)
        result = pd.DataFrame(cur.fetchall())
result

,0,1,2
0,Darren,Smith,2012-09-26 18:08:45


## Задание 3 (2 балла)

Составьте список времени начала бронирования теннисных кортов на дату "2012-09-21".  
В итоговой выборке верните список пар времени начала и названий объектов, упорядоченных по времени.

In [120]:
connection = psycopg2.connect(
    host='ep-bitter-breeze-a2oczi1e.eu-central-1.aws.neon.tech',
    database='club',
    user='KotelnikovNA',
    password='9YRzhyH2txkK'
    )

sql = '''
select (to_char(b.starttime, 'HH24:MI:SS'))::time as time, f.name
from club.cd.bookings b 
join club.cd.facilities f 
on f.facid = b.facid
where  ((to_char(b.starttime, 'DD.MM.YYYY')) = '21.09.2012') and (name like 'Tennis Court%')
order by time
'''

with cursor:
    with connection.cursor() as cur:
        cur.execute(sql)
        result = pd.DataFrame(cur.fetchall())
result

,0,1
0,08:00:00,Tennis Court 1
1,08:00:00,Tennis Court 2
2,09:30:00,Tennis Court 1
3,10:00:00,Tennis Court 2
4,11:30:00,Tennis Court 2
5,12:00:00,Tennis Court 1
6,13:30:00,Tennis Court 1
7,14:00:00,Tennis Court 2
8,15:30:00,Tennis Court 1
9,16:00:00,Tennis Court 2


## Задание 4 (2 балла)

 Выведите список всех участников, включая человека, который их рекомендовал (если таковой имеется).  
 Убедитесь, что результаты упорядочены по (фамилия, имя).

In [146]:
connection = psycopg2.connect(
    host='ep-bitter-breeze-a2oczi1e.eu-central-1.aws.neon.tech',
    database='club',
    user='KotelnikovNA',
    password='9YRzhyH2txkK'
    )

sql = '''
select m.firstname ||' '|| m.surname as FIO, m2.firstname||' '|| m2.surname as FIOrec
from club.cd.members m 
left join  club.cd.members m2
on m.recommendedby = m2.memid
order by FIO

'''

with cursor:
    with connection.cursor() as cur:
        cur.execute(sql)
        result = pd.DataFrame(cur.fetchall())

result

        

,0,1
0,Anna Mackenzie,Darren Smith
1,Anne Baker,Ponder Stibbons
2,Burton Tracy,None
3,Charles Owen,Darren Smith
4,Darren Smith,None
5,Darren Smith,None
6,David Farrell,None
7,David Jones,Janice Joplette
8,David Pinker,Jemima Farrell
9,Douglas Jones,David Jones


## Задание 5 (2 балла)

Составьте список чила слотов забронированных по каждой услуге в сентябре 2012ого года.  

В выводе отобразите две колонки: `facid`, `Total Slots`, и отсортируйте по числу слотов.

In [158]:
connection = psycopg2.connect(
    host='ep-bitter-breeze-a2oczi1e.eu-central-1.aws.neon.tech',
    database='club',
    user='KotelnikovNA',
    password='9YRzhyH2txkK'
    )

sql = '''
select facid, sum(slots) as summ_sl 
from club.cd.bookings b 
where ((to_char(b.starttime, 'MM.YYYY')) = '09.2012')
group by facid
order by summ_sl

'''

with cursor:
    with connection.cursor() as cur:
        cur.execute(sql)
        result = pd.DataFrame(cur.fetchall())

result


,0,1
0,5,122
1,3,422
2,7,426
3,8,471
4,6,540
5,2,570
6,1,588
7,0,591
8,4,648


## Задание 6 (2 балла)

Составьте список бронирований на день 2012-09-14, которые обойдутся участнику (или гостю) более чем в 30 долларов. Помните, что у гостя всегда идентификатор 0 и расходы гостей отличаются от расходов участников (расходы указаны за получасовой слот).

В выводе отобразите три колонки: `member`, `facility`, `cost` и отсортируйте по убыванию стоимости. В колонке `member` объединим имя и фамилию следующим образом:

`mems.firstname || ' ' || mems.surname as member`  
(в данном случае `mems` — это алиас на колонку `members`)

In [185]:
connection = psycopg2.connect(
    host='ep-bitter-breeze-a2oczi1e.eu-central-1.aws.neon.tech',
    database='club',
    user='KotelnikovNA',
    password='9YRzhyH2txkK'
    )

sql = '''
select* 
from (select (m.firstname || ' ' || m.surname) as member, f.name,
(case 
when b.memid = 0 then b.slots * f.guestcost
when b.memid <> 0 then b.slots * f.membercost
end ) as cost
from club.cd.bookings b 
join club.cd.facilities f 
on b.facid=f.facid
join club.cd.members m 
on b.memid=m.memid
where ((to_char(b.starttime, 'DD.MM.YYYY')) = '14.09.2012')
) as foo
where cost > 30
order by cost desc



'''

with cursor:
    with connection.cursor() as cur:
        cur.execute(sql)
        result = pd.DataFrame(cur.fetchall())

result


,0,1,2
0,GUEST GUEST,Massage Room 2,320
1,GUEST GUEST,Massage Room 1,160
2,GUEST GUEST,Massage Room 1,160
3,GUEST GUEST,Massage Room 1,160
4,GUEST GUEST,Tennis Court 2,150
5,Jemima Farrell,Massage Room 1,140
6,GUEST GUEST,Tennis Court 1,75
7,GUEST GUEST,Tennis Court 1,75
8,GUEST GUEST,Tennis Court 2,75
9,Ponder Stibbons,Massage Room 1,70
